In [11]:
import cv2
import mediapipe as mp
import math

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Initialize MediaPipe Hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)

def is_like_gesture(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
    thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    index_pip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]
    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    middle_pip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    ring_pip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    pinky_pip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]

    # Check thumb and other finger positions
    thumb_folded = thumb_tip.y > thumb_ip.y > thumb_mcp.y
    index_folded = index_tip.y > index_pip.y > index_mcp.y
    middle_folded = middle_tip.y > middle_pip.y > middle_mcp.y
    ring_folded = ring_tip.y > ring_pip.y > ring_mcp.y
    pinky_folded = pinky_tip.y > pinky_pip.y > pinky_mcp.y

    # Check thumb and other finger distances
    thumb_pinky_dist = math.sqrt((thumb_tip.x - pinky_tip.x) ** 2 + (thumb_tip.y - pinky_tip.y) ** 2)
    fingers_dist = math.sqrt((index_tip.x - pinky_tip.x) ** 2 + (index_tip.y - pinky_tip.y) ** 2)

    # Check if all conditions for dislike gesture are satisfied
    if thumb_folded and index_folded and middle_folded and ring_folded and pinky_folded and thumb_pinky_dist > fingers_dist:
        return "Dislike"
    
    # Check if all conditions for like gesture are satisfied
    if thumb_folded and index_folded and middle_folded and ring_folded and pinky_folded and thumb_tip.y < index_tip.y:
        return "Like"
    
    return "None"


cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
        
        # Flip the image horizontally for a mirrored display
        image = cv2.flip(image, 1)
        
        # Convert the image color space from BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Process the image and detect hands
        results = hands.process(image_rgb)
        
        # Check if hands are detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the image
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Check if the gesture is a like or dislike gesture
                gesture = is_like_gesture(hand_landmarks)
                if gesture == "Dislike":
                    cv2.putText(image, "Dislike", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                elif gesture == "Like":
                    cv2.putText(image, "Like", (250, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 200, 255), 2)
                else:
                    cv2.putText(image, "unknown", (300, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (50, 200, 255), 2)
        
        # Display the image with annotated hand landmarks
        cv2.imshow('Sign Language Detection', image)
        
        # Exit loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()